##Mounting drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/My\ Drive/Thesis/Notebooks
%ls

/content/drive/My Drive/Thesis/Notebooks
 Classifier-Gene-2.ipynb
 Classifier-Gene-2_kernel_width_test.ipynb
 Classifier-Gene.ipynb
 Classifier-Normal-2.ipynb
 Classifier-Normal-2-Kernel_width-Testing.ipynb
 Classifier-Normal.ipynb
'Copy1 of Classifier-Normal-2.ipynb'
'Copy2 of Classifier-Normal-2.ipynb'
'Copy3 of Classifier-Normal-2.ipynb'
 IRelief2.ipynb
 IRelief.ipynb
 Load_dataset.ipynb
 Modified_MultiSurf.ipynb
 Modified_Overlapping_MultiSurf.ipynb
 MultiSurf.ipynb
 Ovelapping_ReliefF.ipynb
 Overlapping_MultiSurf.ipynb
 ReliefF.ipynb
 Relief.ipynb
 RFS.ipynb


##Installing Libraries

In [ ]:
!pip install ipynb
!pip install skrebate

  Using cached https://files.pythonhosted.org/packages/31/42/4c0bbb66390e3a68e04ebf134c8d074a00c18b5882293f8ace5f7497fbf0/ipynb-0.5.1-py3-none-any.whl


##Importing Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, RepeatedKFold, RepeatedStratifiedKFold, StratifiedKFold, KFold, LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_recall_fscore_support
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from skrebate import ReliefF, MultiSURF, SURF, SURFstar, MultiSURFstar
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from ipynb.fs.full.Load_dataset import load_dataset, get_dataset_names, load_gene_dataset, get_gene_dataset_names
from ipynb.fs.full.ReliefF import ReliefF_configure
from ipynb.fs.full.MultiSurf import MultiSurf_configure
from ipynb.fs.full.Modified_MultiSurf import Modified_MultiSurf_configure
from ipynb.fs.full.RFS import RFS_configure
from ipynb.fs.full.Relief import Relief_configure
from ipynb.fs.full.IRelief import I_Relief_configure
from ipynb.fs.full.IRelief2 import I_Relief2_configure
from ipynb.fs.full.Overlapping_MultiSurf import Overlapping_MultiSurf_configure
from ipynb.fs.full.Modified_Overlapping_MultiSurf import Modified_Overlapping_MultiSurf_configure
from ipynb.fs.full.Ovelapping_ReliefF import Ovelapping_ReliefF_configure
from os import path
import time

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
dataset: waveform
instances = 5000, features= 21 
[0 1] [0.4        0.11818182]
[1 0] [-0.12702534 -0.1031529 ]


In [ ]:
folderPath = '/content/drive/My Drive/Thesis/Results'
randomState = 123
plt.rc('xtick',labelsize=15)
plt.rc('ytick',labelsize=15)

plotStyles = {
      'markers' : ["o","^","*","s","P","X","d", "p", "H","D"],
      'color' : ['#273c75', '#6c5ce7', '#d63031', '#00b894','#00cec9',  '#0984e3', '#74b9ff', '#6ab04c', '#e056fd', '#f9ca24'],
      'lineStyle' : 'solid',
      'markerSize' :  10,
  }

## Load Dataset

In [ ]:
def dataset_preparetion(iterationNum,dName, dataset_dict, noisePercentage = 0, datasetType= 'normal'):
  global randomState, folderPath
  print('datasetType= ',datasetType)
  X = dataset_dict['attributes']
  Y = dataset_dict['target']
  categoricalX = dataset_dict['categoricalX']
  trainSize = dataset_dict['trainSize']
  testSize = dataset_dict['testSize']
  # print("Before : ", X.shape, Y.shape)

  # Adding irrelavant Features
  mu, sigma = 0, 5 # mean and standard deviation
  np.random.seed(randomState)
  irrelevantFeatures = np.random.normal(mu, sigma, (X.shape[0], 50))
  X = np.concatenate((X, irrelevantFeatures), axis=1)
  # print("After : ", X.shape, Y.shape)  
  
  XTrain, XTest, YTrain, YTest = train_test_split(X, Y, train_size = trainSize, test_size= testSize, stratify=Y, random_state = randomState + iterationNum)

  # Adding noise i.e. misslabeling training data
  if noisePercentage > 0:
    classes = np.unique(YTrain)

    numOfMislabels = np.floor(len(YTrain) * (noisePercentage/100)).astype(int)
    # print(numOfMislabels)

    # Choose the random instances
    rndIdx = np.random.choice(len(YTrain), numOfMislabels, replace=False)   # random indices of m instances from total of n instances
    # print(rndIdx)

    for idx in rndIdx:
      restOfTheClasses = classes[classes != YTrain[idx]]
      YTrain[idx] = restOfTheClasses[np.random.choice(len(restOfTheClasses), 1, replace=False).item()]
      # print("before : ", Y[idx])
      # print('after : ', newLabel) 

  return XTrain, XTest, YTrain, YTest, categoricalX

##Preprocessing and Feature Selection

In [ ]:
def feature_selection(X, Y, methods, K_RFS = 7, K_ReliefF = 10, theta_RFS = 3, numUpdates='all', numOfIterations= 5, prior='uniform', categoricalX='off', kernelWidth = 5, theta_IRelief = 1e-9): 
  cache = {}
  if 'RFS' in methods:
    cache['RFS'] = RFS_configure(X,Y, K = K_RFS, theta= theta_RFS)
  
  isBinary = len(np.unique(Y)) == 2
  if 'Relief' in methods:
    if isBinary:
       cache['Relief'] = Relief_configure(X,Y, numUpdates=numUpdates, categoricalX=categoricalX)
  
  if 'ReliefF' in methods:
    start = time.time()
    cache['ReliefF'] = ReliefF_configure(X, Y, K=K_ReliefF, prior=prior, numUpdates=numUpdates, categoricalX=categoricalX) 
    print('ReliefF run time= ',time.time()-start)
 
  if 'IRelief' in methods:
    start = time.time()
    cache['IRelief'] = I_Relief_configure(X, Y, numOfIterations=numOfIterations, categoricalX = categoricalX, kernelWidth = kernelWidth, theta = theta_IRelief, prior=prior)
    print('IRelief run time= ',time.time()-start)

  if 'IRelief2' in methods:
    start = time.time()
    cache['IRelief2'] = I_Relief2_configure(X, Y, numOfIterations=numOfIterations, categoricalX = categoricalX, kernelWidth = kernelWidth, theta = theta_IRelief)
    print('IRelief2 run time= ',time.time()-start)

  if 'MultiSurf' in methods:
    start = time.time()
    cache['MultiSurf'] = MultiSurf_configure(X,Y,prior=prior, numUpdates=numUpdates, categoricalX=categoricalX)
    print('MultiSurf run time= ',time.time()-start)

  if 'Modified_MultiSurf' in methods:
    start = time.time()
    cache['Modified_MultiSurf'] = Modified_MultiSurf_configure(X,Y,prior=prior, numUpdates=numUpdates, categoricalX=categoricalX, kernelWidth = kernelWidth)
    print('Modified_MultiSurf run time= ',time.time()-start)

  if 'Overlapping_MultiSurf' in methods:
    start = time.time()
    cache['Overlapping_MultiSurf'] = Overlapping_MultiSurf_configure(X,Y,prior=prior, numUpdates=numUpdates, categoricalX=categoricalX, kernelWidth = kernelWidth)
    print('Overlapping_MultiSurf run time= ',time.time()-start)

  if 'Modified_Overlapping_MultiSurf' in methods:
    start = time.time()
    cache['Modified_Overlapping_MultiSurf'] = Modified_Overlapping_MultiSurf_configure(X,Y,prior=prior, numUpdates=numUpdates, categoricalX=categoricalX, kernelWidth = kernelWidth)
    print('Modified_Overlapping_MultiSurf run time= ',time.time()-start)

  # for kernel widh choose **********************************************************************
  if 'Modified_Overlapping_MultiSurf_0.05' in methods:
    start = time.time()
    cache['Modified_Overlapping_MultiSurf_0.05'] = Modified_Overlapping_MultiSurf_configure(X,Y,prior=prior, numUpdates=numUpdates, categoricalX=categoricalX, kernelWidth = 0.05)
    print('Modified_Overlapping_MultiSurf_0.05 run time= ',time.time()-start)

  if 'Modified_Overlapping_MultiSurf_0.3' in methods:
    start = time.time()
    cache['Modified_Overlapping_MultiSurf_0.3'] = Modified_Overlapping_MultiSurf_configure(X,Y,prior=prior, numUpdates=numUpdates, categoricalX=categoricalX, kernelWidth = 0.3)
    print('Modified_Overlapping_MultiSurf_0.3 run time= ',time.time()-start)

  if 'Modified_Overlapping_MultiSurf_1' in methods:
    start = time.time()
    cache['Modified_Overlapping_MultiSurf_1'] = Modified_Overlapping_MultiSurf_configure(X,Y,prior=prior, numUpdates=numUpdates, categoricalX=categoricalX, kernelWidth = 1)
    print('Modified_Overlapping_MultiSurf_1 run time= ',time.time()-start)

  if 'Modified_Overlapping_MultiSurf_3' in methods:
    start = time.time()
    cache['Modified_Overlapping_MultiSurf_3'] = Modified_Overlapping_MultiSurf_configure(X,Y,prior=prior, numUpdates=numUpdates, categoricalX=categoricalX, kernelWidth = 3)
    print('Modified_Overlapping_MultiSurf_3 run time= ',time.time()-start)

  if 'Modified_Overlapping_MultiSurf_5' in methods:
    start = time.time()
    cache['Modified_Overlapping_MultiSurf_5'] = Modified_Overlapping_MultiSurf_configure(X,Y,prior=prior, numUpdates=numUpdates, categoricalX=categoricalX, kernelWidth = 5)
    print('Modified_Overlapping_MultiSurf_5 run time= ',time.time()-start)

  if 'Modified_Overlapping_MultiSurf_10' in methods:
    start = time.time()
    cache['Modified_Overlapping_MultiSurf_10'] = Modified_Overlapping_MultiSurf_configure(X,Y,prior=prior, numUpdates=numUpdates, categoricalX=categoricalX, kernelWidth = 10)
    print('Modified_Overlapping_MultiSurf_10 run time= ',time.time()-start)

  # *********************************************************************************************
  if 'Overlapping_ReliefF' in methods:
    start = time.time()
    cache['Overlapping_ReliefF'] = Ovelapping_ReliefF_configure(X,Y,K=K_ReliefF,prior=prior, numUpdates=numUpdates, categoricalX=categoricalX, kernelWidth=kernelWidth)
    print('Overlapping_ReliefF run time= ',time.time()-start)

  if 'ReBATE_ReliefF' in methods:
    start = time.time()
    reliefF = ReliefF(n_features_to_select=min(X.shape[1], 1000), n_neighbors=K_ReliefF)
    reliefF.fit(X, Y)
    weight = reliefF.feature_importances_
    ranked = reliefF.top_features_
    cache['ReBATE_ReliefF'] = ranked, weight
    print('ReBATE_ReliefF run time= ',time.time()-start)

  if 'ReBATE_SURF' in methods:
    start = time.time()
    surf = SURF(n_features_to_select=min(X.shape[1], 1000))
    surf.fit(X, Y)
    weight = surf.feature_importances_
    ranked = surf.top_features_
    cache['ReBATE_SURF'] = ranked, weight
    print('ReBATE_SURF run time= ',time.time()-start)

  if 'ReBATE_SURFstar' in methods:
    start = time.time()
    surfStar = SURFstar(n_features_to_select=min(X.shape[1], 1000))
    surfStar.fit(X, Y)
    weight = surfStar.feature_importances_
    ranked = surfStar.top_features_
    cache['ReBATE_SURFstar'] = ranked, weight
    print('ReBATE_SURFstar run time= ',time.time()-start)

  if 'ReBATE_MultiSURFstar' in methods:
    start = time.time()
    multiSurfStar = MultiSURFstar(n_features_to_select=min(X.shape[1], 1000))
    multiSurfStar.fit(X, Y)
    weight = multiSurfStar.feature_importances_
    ranked = multiSurfStar.top_features_
    cache['ReBATE_MultiSURFstar'] = ranked, weight
    print('ReBATE_MultiSURFstar run time= ',time.time()-start)

  if 'ReBATE_MultiSURF' in methods:
    start = time.time()
    multiSurf = MultiSURF(n_features_to_select=min(X.shape[1], 1000))
    multiSurf.fit(X, Y)
    weight = multiSurf.feature_importances_
    ranked = multiSurf.top_features_
    cache['ReBATE_MultiSURF'] = ranked, weight
    print('ReBATE_MultiSURF run time= ',time.time()-start)  

  return cache

In [ ]:
# print('ranked')
# print(ranked)
# print('Rvalue')
# print(Rvalue[np.array(ranked, dtype=int)])

In [ ]:
# print('ranked')
# print(reliefF_ranked)
# print('reliefF_weight')
# print(reliefF_weight[np.array(reliefF_ranked, dtype=int)])

##Feature Scaling

In [ ]:
def feature_scaling(XTrain, XTest):
  scaler = StandardScaler()
  scaler.fit(XTrain)
  # print("Mean :",scaler.mean_, "Var :", scaler.var_)

  XTrain = scaler.transform(XTrain)
  XTest = scaler.transform(XTest)

  return XTrain, XTest, scaler

##KNN classification

In [ ]:
def KNN_classifier(XTrain, YTrain, XTest, YTest, Num_neighbors=7):
  classifier = KNeighborsClassifier(n_neighbors=Num_neighbors)
  classifier.fit(XTrain, YTrain)  
  Y_pred = classifier.predict(XTest)
  score = accuracy_score(YTest, Y_pred)
  # score1 = classifier.score(XTest, YTest)
  # print(score,score1)
  
  return Y_pred, score

##SVM classification

In [ ]:
def SVM_classifier(XTrain, YTrain, XTest, YTest, kernel='linear', degree=3):
  # degree is for polynomial kernel
  svclassifier = SVC(kernel=kernel)
  svclassifier.fit(XTrain, YTrain)
  Y_pred = svclassifier.predict(XTest)
  score = accuracy_score(YTest, Y_pred)

  return Y_pred, score

##Evaluataion

In [ ]:
def evaluation(YTest, Y_pred):
  print(confusion_matrix(YTest, Y_pred))
  print(classification_report(YTest, Y_pred))

##Classification

In [ ]:
def classification(XTrain, XTest, YTrain, YTest, ranked, algo, featureRange, datasetName, Num_neighbors=7, kernel='linear', degree=3):

  scoreCollection = np.array([])
  YPredCollection = []
  XTrain, XTest, _ = feature_scaling(XTrain, XTest)

  for n in featureRange:
    feature_idx = np.array(ranked[0:n])
    
    # if np.sum(feature_idx>=XTrain.shape[1]-50):
    #   print('irrelevent selected: ',feature_idx)
    # print(feature_idx)
    
    # print(feature_idx)
    scores = np.array([])
    XSubTrain, XSubTest = XTrain[:, feature_idx], XTest[:, feature_idx]
    
    # print("XSubTRAIN:", XSubTrain.shape, "XSubTEST:", XSubTest.shape)
    # print("YTRAIN:", YTrain.shape, "YTEST:", YTest.shape)

    if algo == 'KNN':
      YPred, score = KNN_classifier(XSubTrain, YTrain, XSubTest, YTest,  Num_neighbors=Num_neighbors)
    else:
      YPred, score = SVM_classifier(XSubTrain, YTrain, XSubTest, YTest, kernel=kernel, degree=degree)

    scoreCollection = np.append(scoreCollection, score)
    YPredCollection.append(YPred)

  # print(datasetName, scoreCollection)
  return np.array(YPredCollection), scoreCollection

## Run and Save results

In [ ]:
def performanceFileWrite(metricName, arrayCollection, index, featureRange, dName, classifierName, isNoisy ): 
  global folderPath 
  filePath = ""
  if isNoisy:
    filePath = folderPath  +  '/Gene/Performance/{}/{}_{}_withNoise.csv'.format(metricName, dName, classifierName)
  else:
    filePath = folderPath  +  '/Gene/Performance/{}/{}_{}_withoutNoise.csv'.format(metricName, dName, classifierName)
  if path.exists(filePath):
    df = pd.read_csv(filePath, sep=",", index_col=['Method', 'Classes'])
    for i in range(index.to_series().nunique()):
      df.loc[index[i],:] = arrayCollection[i] 
  else:
    df = pd.DataFrame(arrayCollection, index=index, columns=featureRange)
    df = df.rename_axis('#features', axis='columns')
 
  print("{} for the dataset {} and classifier {} (Noisy = {})".format(metricName ,dName, classifierName, isNoisy))
  print(df)
  df.to_csv(filePath, sep=",")

In [ ]:
def performanceReport(methods, YActual, YPredDict, featureRange, dName, isNoisy, classifierName):
  
  truePosPlusSupportArrayCollection = []
  precisionArrayCollection, recallArrayCollection, f1scoreArrayCollection = [], [], []
  for method in methods: 
    truePosPlusSupportDict = {}
    precisionDict, recallDict, f1scoreDict = {}, {}, {}
    for YPred in YPredDict[method]:
      confMat = confusion_matrix(YActual, YPred)
      precision, recall, f1score, _ = precision_recall_fscore_support(YActual, YPred, average=None)
      truePositives = np.diag(confMat) 
      support = np.sum(confMat, axis = 1) 
      for i in range(len(truePositives)):
        if i not in truePosPlusSupportDict:
          truePosPlusSupportDict[i] = np.array(['A={},TP={}'.format(support[i], truePositives[i])])
          precisionDict[i] = np.array(precision[i])
          recallDict[i] = np.array(recall[i])
          f1scoreDict[i] = np.array(f1score[i])
        else:
          truePosPlusSupportDict[i] = np.append(truePosPlusSupportDict[i], 'A={},TP={}'.format(support[i], truePositives[i]))
          precisionDict[i] = np.append(precisionDict[i], precision[i])
          recallDict[i] = np.append(recallDict[i], recall[i])
          f1scoreDict[i] = np.append(f1scoreDict[i], f1score[i])

    truePosPlusSupportArray = []
    precisionArray, recallArray, f1scoreArray = [], [], []
    for key in truePosPlusSupportDict:
      truePosPlusSupportArray.append(truePosPlusSupportDict[key])
      precisionArray.append(precisionDict[key])
      recallArray.append(recallDict[key])
      f1scoreArray.append(f1scoreDict[key])
    truePosPlusSupportArrayCollection = truePosPlusSupportArrayCollection + truePosPlusSupportArray
    precisionArrayCollection = precisionArrayCollection + precisionArray
    recallArrayCollection = recallArrayCollection + recallArray
    f1scoreArrayCollection = f1scoreArrayCollection + f1scoreArray

  truePosPlusSupportArrayCollection = np.array(truePosPlusSupportArrayCollection)
  precisionArrayCollection = np.array(precisionArrayCollection)
  recallArrayCollection = np.array(recallArrayCollection)
  f1scoreArrayCollection = np.array(f1scoreArrayCollection)
  
  classes = np.unique(YActual)
  # Confusion matrix
  index = pd.MultiIndex.from_product([methods, classes], names=['Method', 'Classes'])

  performanceFileWrite('ConfusionMatrix', truePosPlusSupportArrayCollection, index, featureRange, dName, classifierName, isNoisy )
  performanceFileWrite('Precision', precisionArrayCollection, index, featureRange, dName, classifierName, isNoisy )
  performanceFileWrite('Recall', recallArrayCollection, index, featureRange, dName, classifierName, isNoisy )
  performanceFileWrite('F1score', f1scoreArrayCollection, index, featureRange, dName, classifierName, isNoisy )

### Feature weight calculating

In [ ]:
def run_and_save_feature_weights(methods, dName, datasetType='gene', numOfFolds = 10, numOfRepeats = 5, noisePercentage=0, numOfPoints = 30, firstEndPoint = 500, secondEndPoint = 2000):
  global randomState, folderPath

  # print('in run_and_save_output datasetType= ',datasetType)
  isNoisy = noisePercentage > 0

  dataset_dict =   load_dataset(dName) if (datasetType == 'normal') else load_gene_dataset(dName) 
  # print('datasetType= ',datasetType)

  X = dataset_dict['attributes']
  Y = dataset_dict['target']
  categoricalX = dataset_dict['categoricalX'] 
  featureNames = dataset_dict['featureNames']
  
  # leaveOneOut = LeaveOneOut()
  # numOfsplits = leaveOneOut.get_n_splits(X)
  repeatedKFold = RepeatedKFold(n_splits = numOfFolds, n_repeats = numOfRepeats, random_state=randomState)
  numOfsplits = repeatedKFold.get_n_splits(X)

  weightsOfMethods = {}
  ranksOfMethods = {}

 
  minItrSaved = 1e9
  dfPrevWeights, dfPrevRanks= {}, {}
  # Load the pre saved iteration for each methods
  for method in methods:
    weigthFileExists = path.exists(folderPath  +  '/Gene/Weights/{}/{}.csv'.format(dName, method))
    rankFileExists = path.exists(folderPath  +  '/Gene/Ranks/{}/{}.csv'.format(dName, method))
    if weigthFileExists:
      dfPrevWeights[method] = pd.read_csv(folderPath  +  '/Gene/Weights/{}/{}.csv'.format(dName, method), sep=",",index_col="Iterations", header=0)
      minItrSaved = min(minItrSaved, dfPrevWeights[method].index.shape[0]-1)
    else:
      minItrSaved = 0

    if rankFileExists:
      dfPrevRanks[method] = pd.read_csv(folderPath  +  '/Gene/Ranks/{}/{}.csv'.format(dName, method), sep=",",index_col="Iterations", header=0)
      minItrSaved = min(minItrSaved, dfPrevRanks[method].index.shape[0]-1)
    else:
      minItrSaved = 0

  iterationNo = -1
  for trainIndex, testIndex in repeatedKFold.split(X):
    iterationNo+=1
    # print("TRAIN:", trainIndex, "TEST:", testIndex)
    XTrain, XTest, YTrain, YTest = X[trainIndex], X[testIndex], Y[trainIndex], Y[testIndex]

    # Here if we estimate best K for KNN classification using 10 fold stratified cross validation, it will help us to do comparison more fairly.
    # Here If we estimate best kernelWidth sigma for I-Relief using 10 fold stratified cross validation, it will help us to do comparison more fairly. 
    # Here If we estimate best K(i.e. number of nearest hits and misses) for Relief-F using 10 fold stratified cross validation, it will help us to do comparison more fairly. 


    # performing feture selection 
    # Using kernelwidth sigma = 3 and number of iterations = 20 and convergence_threshold = 1e-5 for I-Relief-1 and I-Relief-2
    # Using number of nearest hits and misses i.e. K = 10 for ReliefF and also using all the training data for Relief and ReliefF
    # Using number of nearest Neighbours k = 7 and overlapping_threshold = 3 for RFS
    # print('Training data size: training examples = {}, features= {}'.format(XTrain.shape[0], XTrain.shape[1]))
    
    cache ={}
    if iterationNo>=minItrSaved:
      # Feature Selection
      cache = feature_selection(XTrain, YTrain, methods, K_RFS = 7, K_ReliefF = 10, theta_RFS = 3, numUpdates='all', numOfIterations = 20, prior='empirical', categoricalX='off', kernelWidth = 5, theta_IRelief = 1e-5)
  
    for method in methods:
      ranked, weight= None, None
      if iterationNo>=minItrSaved:
        ranked, weight = cache[method]        
      else:
        ranked, weight = dfPrevRanks[method].loc[iterationNo,:].to_numpy(), dfPrevWeights[method].loc[iterationNo,:].to_numpy()

      if method not in weightsOfMethods:
        weightsOfMethods[method] = [weight]
        ranksOfMethods[method] = [ranked]
      else:
        weightsOfMethods[method].append(weight)
        ranksOfMethods[method].append(ranked)

      if iterationNo == minItrSaved:
        print('iterationNo = ', iterationNo)
        dfWeights, dfRanks = None, None
        if featureNames.shape[0] == 0:
          dfWeights = pd.DataFrame(np.array(weightsOfMethods[method]))
          dfRanks = pd.DataFrame(np.array(ranksOfMethods[method]))
        else:
          dfWeights = pd.DataFrame(np.array(weightsOfMethods[method]), columns=featureNames)
          dfRanks = pd.DataFrame(np.array(ranksOfMethods[method]), columns=featureNames)

        dfWeights = dfWeights.rename_axis('Iterations', axis='index').rename_axis('Features', axis = 'columns')
        dfWeights.to_csv(folderPath + '/Gene/Weights/{}/{}.csv'.format(dName,method), sep=",")

        dfRanks = dfRanks.rename_axis('Iterations', axis='index').rename_axis('Features', axis = 'columns')
        dfRanks.to_csv(folderPath + '/Gene/Ranks/{}/{}.csv'.format(dName,method), sep=",")
      
      elif iterationNo > minItrSaved:
        print('iterationNo = ', iterationNo)
        newDfWeights = pd.DataFrame(np.array([weight]), index=[iterationNo])
        newDfRanks = pd.DataFrame(np.array([ranked]), index=[iterationNo])

        newDfWeights.to_csv(folderPath + '/Gene/Weights/{}/{}.csv'.format(dName,method), sep=",", mode='a', header = False)
        newDfRanks.to_csv(folderPath + '/Gene/Ranks/{}/{}.csv'.format(dName,method), sep=",", mode='a', header = False) 

  # saving avg weigts of all methods
  avgWeightsOfMethods ={}
  for method in methods:
    weightsOfMethods[method] = np.array(weightsOfMethods[method])
    avgWeights = np.sum(weightsOfMethods[method],axis=0)/numOfsplits
    avgWeightsOfMethods[method]=avgWeights

  dfAvgWeights = None
  if path.exists(folderPath+ '/Gene/Weights/{}/avgWeights.csv'.format(dName)):
    dfAvgWeights = pd.read_csv(folderPath + '/Gene/Weights/{}/avgWeights.csv'.format(dName), sep=",", index_col="Methods", header=0)
    for method in methods:
      dfAvgWeights.loc[method] = avgWeightsOfMethods[method]
  else:
    if featureNames.shape[0]==0:
      dfAvgWeights= pd.DataFrame.from_dict(avgWeightsOfMethods, orient='index')
    else:
      dfAvgWeights= pd.DataFrame.from_dict(avgWeightsOfMethods, orient='index', columns=featureNames)
  dfAvgWeights = dfAvgWeights.rename_axis('Methods', axis='index').rename_axis('Features', axis = 'columns')
  dfAvgWeights.to_csv(folderPath + '/Gene/Weights/{}/avgWeights.csv'.format(dName), sep=",")
  


### Run classification

In [ ]:
def run_and_save_output(methods, dName, datasetType='gene', numOfFolds = 10, numOfRepeats = 5, noisePercentage=0, numOfPoints = 30, firstEndPoint = 500, secondEndPoint = 2000, featureRange = np.array([])):
  global randomState, folderPath
  # print('in run_and_save_output datasetType= ',datasetType)
  isNoisy = noisePercentage > 0

  totalSvmScores = 0
  totalKnnScores = 0

  dataset_dict =   load_dataset(dName) if (datasetType == 'normal') else load_gene_dataset(dName) 
  # print('datasetType= ',datasetType)

  X = dataset_dict['attributes']
  Y = dataset_dict['target']
  categoricalX = dataset_dict['categoricalX'] 

  # numOfPointsFirstRange = int(round(numOfPoints*(2/3)))
  # numOfPointsSecondRange = int(numOfPoints - numOfPointsFirstRange)
  # firstEndPoint = min(X.shape[1], firstEndPoint)
  # firstRange = np.linspace(1, firstEndPoint, numOfPointsFirstRange, dtype=int)

  # secondEndPoint = min(X.shape[1], secondEndPoint)
  # secondRangeStart = firstEndPoint + (secondEndPoint - firstEndPoint)/numOfPointsSecondRange 
  # secondRange = np.linspace(secondRangeStart, secondEndPoint, numOfPointsSecondRange , dtype=int)

  # featureRange = np.unique(np.append(firstRange, secondRange))
  # print(featureRange)

  tempFeatureRange = np.array([1,5,10,20,40,60,80,100,120,140,160,180,200,220,250,280,300,350,400,450,500,750,1000,1250,1500,2000])
  featureRange = featureRange if len(featureRange) != 0 else tempFeatureRange

  # leaveOneOut = LeaveOneOut()
  # numOfsplits = leaveOneOut.get_n_splits(X)
  repeatedKFold = RepeatedKFold(n_splits = numOfFolds, n_repeats = numOfRepeats, random_state=randomState)
  numOfsplits = repeatedKFold.get_n_splits(X)
  YActual = np.array([])
  YPredSVMDict = {}
  YPredKNNDict = {}
  YCollected= False

  svmScoresList = []
  knnScoresList = []
  
  for method in methods:
    df = pd.read_csv(folderPath  +  '/Gene/Ranks/{}/{}.csv'.format(dName, method), sep=",",index_col="Iterations", header=0)
    featureNames = df.columns.to_numpy() #featureNames in an array
    ranks = df.to_numpy() #feature ranks of each iteration in each rows of ranks array

    svmScores = 0
    knnScores = 0
    itrationNo = -1
    for trainIndex, testIndex in repeatedKFold.split(X):
      itrationNo += 1
      # print("TRAIN:", trainIndex, "TEST:", testIndex)
      XTrain, XTest, YTrain, YTest = X[trainIndex], X[testIndex], Y[trainIndex], Y[testIndex]
      if not YCollected:
        YActual = np.append(YActual, YTest) 

      # Here if we estimate best K for KNN classification using 10 fold stratified cross validation, it will help us to do comparison more fairly.
      # Here If we estimate best kernelWidth sigma for I-Relief using 10 fold stratified cross validation, it will help us to do comparison more fairly. 
      # Here If we estimate best K(i.e. number of nearest hits and misses) for Relief-F using 10 fold stratified cross validation, it will help us to do comparison more fairly. 


      # performing feture selection 
      # Using kernelwidth sigma = 5 and number of iterations = 20 and convergence_threshold = 1e-5 for I-Relief-1 and I-Relief-2
      # Using number of nearest hits and misses i.e. K = 10 for ReliefF and also using all the training data for Relief and ReliefF
      # Using number of nearest Neighbours k = 7 and overlapping_threshold = 3 for RFS
      # print('Training data size: training examples = {}, features= {}'.format(XTrain.shape[0], XTrain.shape[1]))

      svmYPred, svmScore = classification(XTrain, XTest, YTrain, YTest, ranks[itrationNo], algo = 'SVM', featureRange = featureRange, datasetName = dName, kernel='linear', degree=3)
      knnYPred, knnScore = classification(XTrain, XTest, YTrain, YTest, ranks[itrationNo], algo = 'KNN', featureRange = featureRange, datasetName = dName, Num_neighbors=7)
      svmScores += svmScore
      knnScores += knnScore  
      if method not in YPredSVMDict:
        YPredSVMDict[method] = svmYPred
        YPredKNNDict[method] = knnYPred
      else:
        YPredSVMDict[method] = np.append(YPredSVMDict[method], svmYPred, axis = 1)
        YPredKNNDict[method] = np.append(YPredKNNDict[method], knnYPred, axis = 1)
    
    svmScoresList.append(svmScores)  #summation of svmScores for each method is appending in svmScoresList
    knnScoresList.append(knnScores)  #summation of knnScores for each method is appending in knnScoresList
    YCollected= True
    

  avgSvmScores = np.array(svmScoresList)/numOfsplits
  avgKnnScores = np.array(knnScoresList)/numOfsplits

  classifiers = np.array(['KNN', 'SVM'])
  # print(classifiers.shape, methods.shape)
  # print(svmScores.shape, knnScores.shape)
  allScores = np.concatenate((avgKnnScores, avgSvmScores), axis=0)
  index = pd.MultiIndex.from_product([classifiers, methods], names=['Classifier', 'Method'])

  filePath = ""
  if isNoisy:
    filePath = folderPath  +  '/Gene/ClassificationAccuracy/{}_withNoise.csv'.format(dName)
  else:
    filePath = folderPath  +  '/Gene/ClassificationAccuracy/{}_withoutNoise.csv'.format(dName)
  
  if path.exists(filePath):
    df = pd.read_csv(filePath, sep=",", index_col=['Classifier','Method'])
    for i in range(index.to_series().nunique()):
      df.loc[index[i],:] = allScores[i] 
  else:
    df = pd.DataFrame(allScores, index=index, columns=featureRange)
    df = df.rename_axis('#features', axis='columns')

  print("Dataset Name :", dName)    
  print(df)
  df.to_csv(filePath, sep=",")
  
  # Writing precision, recall, f1-score, support to the file
  performanceReport(methods, YActual, YPredSVMDict, featureRange, dName, isNoisy, 'SVM')
  performanceReport(methods, YActual, YPredKNNDict, featureRange, dName, isNoisy, 'KNN')

In [ ]:
# def run_and_save_output(dName, datasetType='gene', noisePercentage=0, numOfPoints = 30, firstEndPoint = 500, secondEndPoint = 2000):
#   global randomState
#   # print('in run_and_save_output datasetType= ',datasetType)
#   isNoisy = noisePercentage > 0

#   totalSvmScores = 0
#   totalKnnScores = 0

#   dataset_dict =   load_dataset(dName) if (datasetType == 'normal') else load_gene_dataset(dName) 
#   # print('datasetType= ',datasetType)

#   X = dataset_dict['attributes']
#   Y = dataset_dict['target']
#   categoricalX = dataset_dict['categoricalX'] 
  
#   # leaveOneOut = LeaveOneOut()
#   # numOfsplits = leaveOneOut.get_n_splits(X)
#   repeatedKFold = KFold(n_splits=10)
#   numOfsplits = repeatedKFold.get_n_splits(X)
#   YActual = np.array([])
#   YPredSVMDict = {}
#   YPredKNNDict = {}

#   for trainIndex, testIndex in repeatedKFold.split(X):
#     # print("TRAIN:", trainIndex, "TEST:", testIndex)
#     XTrain, XTest, YTrain, YTest = X[trainIndex], X[testIndex], Y[trainIndex], Y[testIndex]
#     YActual = np.append(YActual, YTest) 

#     # Here if we estimate best K for KNN classification using 10 fold stratified cross validation, it will help us to do comparison more fairly.
#     # Here If we estimate best kernelWidth sigma for I-Relief using 10 fold stratified cross validation, it will help us to do comparison more fairly. 
#     # Here If we estimate best K(i.e. number of nearest hits and misses) for Relief-F using 10 fold stratified cross validation, it will help us to do comparison more fairly. 


#     # performing feture selection 
#     # Using kernelwidth sigma = 3 and number of iterations = 20 and convergence_threshold = 1e-5 for I-Relief-1 and I-Relief-2
#     # Using number of nearest hits and misses i.e. K = 10 for ReliefF and also using all the training data for Relief and ReliefF
#     # Using number of nearest Neighbours k = 7 and overlapping_threshold = 3 for RFS
#     # print('Training data size: training examples = {}, features= {}'.format(XTrain.shape[0], XTrain.shape[1]))
    
#     # Feature Selection
#     cache = feature_selection(XTrain, YTrain, K_RFS = 7, K_ReliefF = 10, theta_RFS = 3, numUpdates='all', numOfIterations = 20, prior='empirical', categoricalX='off', kernelWidth = 5, theta_IRelief = 1e-5)
    
#     numOfPointsFirstRange = int(round(numOfPoints*(2/3)))
#     numOfPointsSecondRange = int(numOfPoints - numOfPointsFirstRange)
#     firstEndPoint = min(XTrain.shape[1], firstEndPoint)
#     firstRange = np.linspace(1, firstEndPoint, numOfPointsFirstRange, dtype=int)

#     secondEndPoint = min(XTrain.shape[1], secondEndPoint)
#     secondRangeStart = firstEndPoint + (secondEndPoint - firstEndPoint)/numOfPointsSecondRange 
#     secondRange = np.linspace(secondRangeStart, secondEndPoint, numOfPointsSecondRange , dtype=int)

#     featureRange = np.unique(np.append(firstRange, secondRange))
#     print(featureRange)

#     svmScores = []
#     knnScores = []
#     methods = np.array([])

#     for method, rankWeight in cache.items():
#       ranked, weight = rankWeight
#       # print(method, ranked[:min(XTrain.shape[1], 1000)])
#       methods = np.append(methods, method)

#       svmYPred, svmScore = classification(XTrain, XTest, YTrain, YTest, ranked, algo = 'SVM', featureRange = featureRange, datasetName = dName, kernel='linear', degree=3)
#       knnYPred, knnScore = classification(XTrain, XTest, YTrain, YTest, ranked, algo = 'KNN', featureRange = featureRange, datasetName = dName, Num_neighbors=7)
#       svmScores.append(svmScore)
#       knnScores.append(knnScore)

#       if method not in YPredSVMDict:
#         YPredSVMDict[method] = svmYPred
#         YPredKNNDict[method] = knnYPred
#       else:
#         YPredSVMDict[method] = np.append(YPredSVMDict[method], svmYPred, axis = 1)
#         YPredKNNDict[method] = np.append(YPredKNNDict[method], knnYPred, axis = 1)

#     svmScores = np.array(svmScores)
#     knnScores = np.array(knnScores)
#     totalSvmScores = totalSvmScores + svmScores
#     totalKnnScores = totalKnnScores + knnScores

#   avgSvmScores = totalSvmScores/numOfsplits
#   avgKnnScores = totalKnnScores/numOfsplits

#   classifiers = np.array(['KNN', 'SVM'])
#   # print(classifiers.shape, methods.shape)
#   # print(svmScores.shape, knnScores.shape)
#   index = pd.MultiIndex.from_product([classifiers, methods], names=['Classifier', 'Method'])
#   df = pd.DataFrame(np.concatenate((avgKnnScores, avgSvmScores), axis=0), index=index, columns=featureRange)
#   df = df.rename_axis('#features', axis='columns')
#   print("Dataset Name :", dName)
#   print(df)
#   if isNoisy:
#     df.to_csv(folderPath  +  '/Gene/ClassificationAccuracy/{}_withNoise.csv'.format(dName), sep=",")
#   else:
#     df.to_csv(folderPath  +  '/Gene/ClassificationAccuracy/{}_withoutNoise.csv'.format(dName), sep=",")

#   # Writing precision, recall, f1-score, support to the file
#   performanceReport(methods, YActual, YPredSVMDict, featureRange, dName, isNoisy, 'SVM')
#   performanceReport(methods, YActual, YPredKNNDict, featureRange, dName, isNoisy, 'KNN')

##Plotting

In [ ]:
def plotting(mehtods, dName):
  global randomState, folderPath, plotStyles

  # extension = ['withoutNoise', 'withNoise']
  extension = ['withoutNoise']
  plt.figure(figsize=(20,10*len(extension)))

  for i in range(len(extension)):
    df = pd.read_csv(folderPath  +  '/Gene/ClassificationAccuracy/{}_{}.csv'.format(dName, extension[i]), sep=",", index_col=['Classifier','Method'])
    noOfFeatures = np.array(df.columns, dtype=int)
    classifiers = np.unique(df.index.get_level_values('Classifier'))
    noOfClassifiers = len(classifiers)
    # methods = np.unique(df.index.get_level_values('Method'))
    noOfMethods = len(methods)
    lines = []

    for j in range(noOfClassifiers):
      plt.subplot(len(extension), noOfClassifiers, (noOfClassifiers * (i)) + j + 1).set_title('Dataset: {} {}, Classifier : {} '.format(dName, extension[i], classifiers[j]), fontsize=20)
      plt.ylabel('Accuracy', fontsize=20)
      plt.xlabel('#features', fontsize=20)

      takenMethods =methods
      # for k in range(noOfMethods):
      #     takenMethods.append(methods[k])

      labels = []    
      for k in range(len(takenMethods)):
        label=takenMethods[k]
        if label == 'Modified_Overlapping_MultiSurf':
          label = 'mMultiSURF'
        # elif label == 'ReBATE_MultiSURF':
        #   label = 'MultiSURF'
        if label.find('ReBATE_')!=-1:
          label= label.replace('ReBATE_','')
        labels.append(label)

        line,  = plt.plot(np.arange(len(noOfFeatures)), df.loc[(classifiers[j], takenMethods[k])], color=plotStyles['color'][k], linestyle=plotStyles['lineStyle'], 
                  marker=plotStyles['markers'][k], markersize=plotStyles['markerSize'], label=label)
        lines.append(line)
          
      plt.xticks(np.arange(len(noOfFeatures)), noOfFeatures,  rotation='vertical')
      
      # # locs, labels = plt.xticks()
      # every_nth = 3
      # for n, label in enumerate(plt.xticks()):
      #   if n % every_nth != 0:
      #     label.visible = False

      # locs, lbls = plt.xticks()
      # lbls[1].visible = False
      # plt.xticks(locs, lbls) 
      
      plt.legend(lines, labels, prop={'size': 18})
        
  plt.show()


# Saving Plots

In [ ]:
def save_plot(methods, dName, fileExtension):
  global randomState, folderPath, plotStyles

  # extension = ['withoutNoise', 'withNoise']
  extension = ['withoutNoise']
  plt.figure(figsize=(10,10))

  for i in range(len(extension)):
    df = pd.read_csv(folderPath + '/Gene/ClassificationAccuracy/{}_{}.csv'.format(dName, extension[i]), sep=",", index_col=['Classifier','Method'])
    noOfFeatures = np.array(df.columns, dtype=int)
    classifiers = np.unique(df.index.get_level_values('Classifier'))
    noOfClassifiers = len(classifiers)
    # methods = np.unique(df.index.get_level_values('Method'))
    noOfMethods = len(methods)
    lines = []

    for j in range(noOfClassifiers):
      plt.clf()
      plt.title('Dataset: {} {}, Classifier : {} '.format(dName, extension[i], classifiers[j]), fontsize=20)
      plt.ylabel('Accuracy', fontsize=20)
      plt.xlabel('#features', fontsize=20)
      
      takenMethods =methods
      # for k in range(noOfMethods):
      #     takenMethods.append(methods[k])


      labels = []    
      for k in range(len(takenMethods)):
        label=takenMethods[k]
        if label == 'Modified_Overlapping_MultiSurf':
          label = 'mMultiSURF'
        # elif label == 'ReBATE_MultiSURF':
        #   label = 'MultiSURF'
        if label.find('ReBATE_')!=-1:
          label= label.replace('ReBATE_','')
        labels.append(label)

        line,  = plt.plot(np.arange(len(noOfFeatures)), df.loc[(classifiers[j], takenMethods[k])], color=plotStyles['color'][k], linestyle=plotStyles['lineStyle'], 
                  marker=plotStyles['markers'][k], markersize=plotStyles['markerSize'], label=label)
        lines.append(line)

      plt.xticks(np.arange(len(noOfFeatures)), noOfFeatures,  rotation='vertical')    
      plt.legend(lines, labels, prop={'size': 18})  
      plt.savefig(folderPath + '/Gene/Plots/ClassificationAccuracy/{}/{}/{}.{}'.format(extension[i], classifiers[j], dName, fileExtension)) 


### Methods to Run

In [ ]:
# methods = np.array(['RFS', 'Relief', 'ReliefF', 'IRelief', 'IRelief2', 'MultiSurf', 'Modified_MultiSurf', 'Overlapping_MultiSurf',
#            'Modified_Overlapping_MultiSurf', 'Overlapping_ReliefF', 'ReBATE_ReliefF', 'ReBATE_SURF', 'ReBATE_SURFstar', 
#            'ReBATE_MultiSURFstar', 'ReBATE_MultiSURF'])
# methods = np.array(['Modified_Overlapping_MultiSurf', 'ReliefF',  'ReBATE_ReliefF', 'ReBATE_SURF', 'ReBATE_SURFstar', 
#            'ReBATE_MultiSURFstar', 'ReBATE_MultiSURF', 'Overlapping_MultiSurf'])
methods = np.array(['Modified_Overlapping_MultiSurf', 'ReliefF', 'ReBATE_MultiSURF','ReBATE_SURF' ])

###CNS(7129/60/2) 

In [ ]:
# run_and_save_feature_weights(methods, dName = 'cns', datasetType = 'gene')
# run_and_save_output(methods, dName = 'cns', datasetType = 'gene')
plotting(methods, 'cns')
save_plot(methods, 'cns', 'svg')
save_plot(methods, 'cns', 'png')

### Colon(2000/62/2)

In [ ]:
# run_and_save_feature_weights(methods, dName = 'colon', datasetType = 'gene')
# run_and_save_output(methods, dName = 'colon', datasetType = 'gene')
plotting(methods, 'colon')
save_plot(methods, 'colon', 'svg')
save_plot(methods, 'colon', 'png')

###lymphoma (4026/66/3) 

In [ ]:
# run_and_save_feature_weights(methods, dName = 'lymphoma', datasetType = 'gene')
# run_and_save_output(methods, dName = 'lymphoma', datasetType = 'gene')
plotting(methods, 'lymphoma')
save_plot(methods, 'lymphoma', 'svg')
save_plot(methods, 'lymphoma', 'png')

###mll(12582/72/3)

In [ ]:
# run_and_save_feature_weights(methods, dName = 'mll', datasetType = 'gene')
# run_and_save_output(methods, dName = 'mll', datasetType = 'gene')
plotting(methods, 'mll')
save_plot(methods, 'mll', 'svg')
save_plot(methods, 'mll', 'png')

###srbct(2308/83/4)

In [ ]:
# run_and_save_feature_weights(methods, dName = 'srbct', datasetType = 'gene')
# run_and_save_output(methods, dName = 'srbct', datasetType = 'gene')
plotting(methods, 'srbct')
save_plot(methods, 'srbct', 'svg')
save_plot(methods, 'srbct', 'png')

###Leukemia(7129/72/2)

In [ ]:
# run_and_save_feature_weights(methods, dName = 'leukemia', datasetType = 'gene')
# run_and_save_output(methods, dName = 'leukemia', datasetType = 'gene')
plotting(methods, 'leukemia')
save_plot(methods, 'leukemia', 'svg')
save_plot(methods, 'leukemia', 'png')

###Leukemia3c(7129/72/3) 

In [ ]:
# run_and_save_feature_weights(methods, dName = 'leukemia3c', datasetType = 'gene')
# run_and_save_output(methods, dName = 'leukemia3c', datasetType = 'gene')
plotting(methods, 'leukemia3c')
save_plot(methods, 'leukemia3c', 'svg')
save_plot(methods, 'leukemia3c', 'png')

###Leukemia4c(imbalanced)  (7129/72/4) 

In [ ]:
# run_and_save_feature_weights(methods, dName = 'leukemia4c', datasetType = 'gene')
# run_and_save_output(methods, dName = 'leukemia4c', datasetType = 'gene')
plotting(methods, 'leukemia4c')
save_plot(methods, 'leukemia4c', 'svg')
save_plot(methods, 'leukemia4c', 'png')

# Gene Datasets Large

In [ ]:
featureRange = np.array([1,5,10,20,50,80,100,120,150,180,200,220,250,300,350,400,450,500,750,1000,1250,1500,2000, 2500, 3000,3500, 4000, 4500, 5000])

### merged_GDS3341  (30803 /41/2)

In [ ]:
# run_and_save_feature_weights(methods, dName = 'merged_GDS3341', datasetType = 'gene')
run_and_save_output(methods, dName = 'merged_GDS3341', datasetType = 'gene', featureRange =featureRange)
plotting(methods, 'merged_GDS3341')
save_plot(methods, 'merged_GDS3341', 'svg')
save_plot(methods, 'merged_GDS3341', 'png')

### merged_GDS3610  (14060 /28/2)

In [ ]:
# run_and_save_feature_weights(methods, dName = 'merged_GDS3610', datasetType = 'gene')
run_and_save_output(methods, dName = 'merged_GDS3610', datasetType = 'gene', featureRange =featureRange)
plotting(methods,'merged_GDS3610')
save_plot(methods,'merged_GDS3610', 'svg')
save_plot(methods,'merged_GDS3610', 'png')

### merged_GDS3858  (14051  /34/2)

In [ ]:
# run_and_save_feature_weights(methods, dName = 'merged_GDS3858', datasetType = 'gene')
run_and_save_output(methods, dName = 'merged_GDS3858', datasetType = 'gene', featureRange =featureRange)
plotting(methods, 'merged_GDS3858')
save_plot(methods, 'merged_GDS3858', 'svg')
save_plot(methods, 'merged_GDS3858', 'png')

### merged_GDS4167  (14060  /52/2)

In [ ]:
# run_and_save_feature_weights(methods, dName = 'merged_GDS4167', datasetType = 'gene')
run_and_save_output(methods, dName = 'merged_GDS4167', datasetType = 'gene', featureRange =featureRange)
plotting(methods,'merged_GDS4167')
save_plot(methods,'merged_GDS4167', 'svg')
save_plot(methods,'merged_GDS4167', 'png')

###merged_GDS4168  (16464 /52/2)

In [ ]:
# run_and_save_feature_weights(methods, dName = 'merged_GDS4168', datasetType = 'gene')
run_and_save_output(methods, dName = 'merged_GDS4168', datasetType = 'gene', featureRange =featureRange)
plotting(methods,'merged_GDS4168')
save_plot(methods,'merged_GDS4168', 'svg')
save_plot(methods,'merged_GDS4168', 'png')

### merged_GDS4824  (30803 /21/2)

In [ ]:
# run_and_save_feature_weights(methods, dName = 'merged_GDS4824', datasetType = 'gene')
run_and_save_output(methods, dName = 'merged_GDS4824', datasetType = 'gene', featureRange =featureRange)
plotting(methods, 'merged_GDS4824')
save_plot(methods, 'merged_GDS4824', 'svg')
save_plot(methods, 'merged_GDS4824', 'png')

### merged_GDS5306  (32389  /38/2)

In [ ]:
featureRange = np.array([1,5,10,20,40,60,100,120,160,200,220,300,400,450,500,750,1000,1250,1500,2000])
# run_and_save_feature_weights(methods, dName = 'merged_GDS5306', datasetType = 'gene')
run_and_save_output(methods, dName = 'merged_GDS5306', datasetType = 'gene', featureRange= featureRange)
plotting(methods, 'merged_GDS5306')
save_plot(methods, 'merged_GDS5306', 'svg')
save_plot(methods, 'merged_GDS5306', 'png')

# Unfinished run

### merged_GDS3837  (30779 /120/2)



In [ ]:
# run_and_save_feature_weights(methods, dName = 'merged_GDS3837', datasetType = 'gene')
# run_and_save_output(methods, dName = 'merged_GDS3837', datasetType = 'gene')
# plotting('merged_GDS3837')
# save_plot('merged_GDS3837', 'svg')
# save_plot('merged_GDS3837', 'png')

### merged_GDS4431  (30803  /146/2)

In [ ]:
# run_and_save_feature_weights(methods, dName = 'merged_GDS4431', datasetType = 'gene')
# run_and_save_output(methods, dName = 'merged_GDS4431', datasetType = 'gene')
# plotting('merged_GDS4431')
# save_plot('merged_GDS4431', 'svg')
# save_plot('merged_GDS4431', 'png')

### merged_GSE106291  (21402/235/2)

In [ ]:
# run_and_save_feature_weights(methods, dName = 'merged_GSE106291', datasetType = 'gene')
# run_and_save_output(methods, dName = 'merged_GSE106291', datasetType = 'gene')
# plotting('merged_GSE106291')
# save_plot('merged_GSE106291', 'svg')
# save_plot('merged_GSE106291', 'png')

###lung2(imbalanced)(12600/203/_) 

In [ ]:
# run_and_save_feature_weights(methods, dName = 'lung2', datasetType = 'gene')
# run_and_save_output(methods, dName = 'lung2', datasetType = 'gene')
# plotting('lung2')
# save_plot('lung2', 'svg')
# save_plot('lung2', 'png')

###ovarian(15154/253/_) 

In [ ]:
# run_and_save_feature_weights(methods, dName = 'ovarian', datasetType = 'gene')
# print("feature selection done")
# run_and_save_output(methods, dName = 'ovarian', datasetType = 'gene')
# plotting('ovarian')
# save_plot('ovarian', 'svg')
# save_plot('ovarian', 'png')

## For Statisctical Analysis Purpose

In [ ]:
# datasets = ['cns', 'colon', 'lymphoma', 'leukemia3c', 'leukemia4c', 'leukemia', 'merged_GDS3341', 'merged_GDS5306']
# methods =  np.array(['Modified_Overlapping_MultiSurf', 'ReliefF', 'ReBATE_MultiSURF','ReBATE_SURF' ])
# extension = ['withoutNoise']

# for dName in datasets:
#   statisticalAnalysisPath = "/content/drive/MyDrive/Thesis/Results/Statistical_Analysis/Accuracy/GeneDataset/{}.csv".format(dName)
#   for i in range(len(extension)):
#     df = pd.read_csv(folderPath  +  '/Gene/ClassificationAccuracy/{}_{}.csv'.format(dName, extension[i]), sep=",", index_col=['Classifier','Method'])
#     noOfFeatures = np.array(df.columns, dtype=int) 

#     scoreCollection = []
#     labels = []
#     for k in range(len(methods)):
#       score = df.loc[('KNN', methods[k])]
#       scoreCollection.append(score.to_numpy())
      
#       label = methods[k]
#       if label == 'Modified_Overlapping_MultiSurf':
#         label = 'mMultiSURF'
#       if label.find('ReBATE_')!=-1:
#         label= label.replace('ReBATE_','')
#       labels.append(label)
    
#     # print(scoreCollection)
#     # print(labels)

#     newDf = pd.DataFrame(np.array(scoreCollection), index=np.array(labels), columns=noOfFeatures)
#     newDf.to_csv(statisticalAnalysisPath, sep=',')

In [ ]:
# datasets = ['cns', 'colon', 'lymphoma', 'leukemia3c', 'leukemia4c', 'leukemia', 'merged_GDS5306']
# methods =  np.array(['Modified_Overlapping_MultiSurf' ])
# extension = ['withoutNoise']
# deletedFeatureRange = np.array(['80', '140','180', '250', '280', '350'])

# scoreCollection = []

# for dName in datasets:
#   statisticalAnalysisPath = "/content/drive/MyDrive/Thesis/Results/Statistical_Analysis/accuracyOfAllGeneDatasets.csv"
#   for i in range(len(extension)):
#     df = pd.read_csv(folderPath  +  '/Gene/ClassificationAccuracy/{}_{}.csv'.format(dName, extension[i]), sep=",", index_col=['Classifier','Method'])
#     score = df.loc[('KNN', 'Modified_Overlapping_MultiSurf')]
#     # print(score)
#     score = score.drop(labels=deletedFeatureRange, errors='ignore')
#     noOfFeatures = np.array(score.index, dtype=int)
#     scoreCollection.append(score)

# newDf = pd.DataFrame(np.array(scoreCollection), index=np.array(datasets), columns=noOfFeatures)
# newDf.to_csv(statisticalAnalysisPath, sep=',')